
**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Edición 2023**

---

# Trabajo práctico entregable - Parte 2

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats


sns.set_context("talk")

## Lectura del dataset

En la notebook 00 se explican los detalles de la siguiente sección.

In [ ]:
url = "https://raw.githubusercontent.com/DiploDatos/AnalisisyVisualizacion/master/sysarmy_survey_2022_processed.csv"
df = pd.read_csv(url)

In [ ]:
df[:3]

In [ ]:
df.profile_gender.unique()

In [ ]:
df.loc[:, "profile_g"] = df.profile_gender.replace(
    {
        "Varón Cis": "Varón cis",
        "Mujer": "Mujer cis",
        "Mujer Cis": "Mujer cis",
        "Femenino": "Mujer cis",
        "mujer": "Mujer cis",
        "Mujer": "Mujer cis",
        "Queer": "Diversidades",
        "Varón Trans": "Diversidades",
        "No binarie": "Diversidades",
        "Mujer Trans": "Diversidades",
        "Fluido": "Diversidades",
        "Bigénero": "Diversidades",
        "Gay": "Diversidades",
    }
).fillna(False)

In [ ]:
df.profile_g.unique()

In [ ]:
df[["profile_g", "salary_monthly_NETO"]].groupby("profile_g").describe()

In [ ]:
df[df.salary_monthly_NETO > 1000][["profile_g", "salary_monthly_NETO"]].groupby(
    "profile_g"
).describe()

In [ ]:
alpha = 0.05

In [ ]:
is_man = df.profile_g == "Varón cis"

groupA = df[(df.salary_monthly_NETO > 1000) & is_man].salary_monthly_NETO
groupB = df[(df.salary_monthly_NETO > 1000) & ~is_man].salary_monthly_NETO

## Ejercicio 1: Estimación

**Consigna:**  Calcular una estimación puntual y un intervalo de confianza de nivel (1-alpha) para la resta entre la media del salario Neto para Hombres menos la media del salario Neto para otros géneros(diferencia de las medias entre el grupoA y grupoB). 
¿Cómo se relaciona este intervalo de confianza con el test de hipótesis?

El intervalo de confianza y el test de hipótesis están estrechamente relacionados porque un intervalo de confianza puede ser utilizado para realizar un test de hipótesis. Por ejemplo, si un intervalo de confianza para la media poblacional no incluye cierto valor especificado, se puede rechazar la hipótesis nula de que la media poblacional es igual a ese valor al nivel de significancia correspondiente al nivel de confianza del intervalo.

In [ ]:
# calcular la media de cada grupo
x1 = np.mean(groupA)
x2 = np.mean(groupB)

print(f"Estimacion puntual de la diferencia de medias {x1-x2}")
# calcular la desviación estándar de cada grupo
s1 = np.std(groupA, ddof=1)
s2 = np.std(groupB, ddof=1)


n1 = len(groupA)
n2 = len(groupB)

s1_sqrd = s1 ** 2
s2_sqrd = s2 ** 2
# calcular el intervalo de confianza del 95% para la diferencia de medias
degrees_of_freedom = round(((s1_sqrd / n1 + s2_sqrd / n2) ** 2) / ( (((s1_sqrd / n1) ** 2) / (n1 - 1)) + ( ((s2_sqrd /n2) ** 2) / (n2 - 1)) ))

t_critico = stats.t.interval(0.95, degrees_of_freedom)[1]
SE = np.sqrt(s1_sqrd / n1 + s2_sqrd / n2)
LI = (x1 - x2) - t_critico*SE
LS = (x1 - x2) + t_critico*SE

print(f"Intervalo de confianza del 95%: ({LI}, {LS})")

In [ ]:
# Estimación puntual
point_estimation = groupA.mean() - groupB.mean()
print(f"Estimación puntual: {point_estimation}")

# Intervalo de confianza
alpha = 0.05
degrees_of_freedom = round(((s1_sqrd / n1 + s2_sqrd / n2) ** 2) / ( (((s1_sqrd / n1) ** 2) / (n1 - 1)) + ( ((s2_sqrd /n2) ** 2) / (n2 - 1)) ))
ci = stats.t.interval(1 - alpha, degrees_of_freedom, loc=point_estimation, scale=np.sqrt(np.var(groupA)/len(groupA) + np.var(groupB)/len(groupB)))
print(f"Intervalo de confianza del 95%: {ci}")

## Ejercicio 2: Test de hipótesis



### 2.1 Formalización

Describir formalmente los distintos componentes de un test de hipótesis para comprobar si la distribución de los salarios es distinta entre los grupos A y B.

**Hipótesis Nula**

$H_0:A=B$

**Estadístico (Pivote)**
  * Identificar el estadístico
  * Escribir qué distribución tiene bajo $H_0$


- Nuestra hipótesis nula será $H_0:A=B$


Para decidir nuestro estadístico, realizamos el test para saber si las varianzas de las muestras pueden considerarse homogéneas.

- Donde nuestra hipótesis nula será $H_0:\sigma_1=\sigma_2$
- El estadístico es $F=\frac{S_x^2/\sigma_x^2}{S_y^2/\sigma_y^2}$, que tiene distribución F con grados de libertad $\nu_x=n_x-1$ y $\nu_y=n_y-1$.
- El estadístico de prueba será $f=\frac{s_x^2}{s_y^2}$.


In [ ]:
# Significancia alpha
alpha=0.001

# Calculo de varianzas muestrales
s2x = np.var(groupA, ddof=1)
s2y = np.var(groupB, ddof=1)

# Calculo de estadístico de prueba
f = s2x / s2y

dof1 = n1 - 1
dof2 = n2 - 1

# F-critical values
f_crit_upper = stats.f.ppf(1 - alpha / 2, dof1, dof2)
f_crit_lower = stats.f.ppf(alpha / 2, dof1, dof2)

print(f"Estadístico de prueba: {f}")
print(f"Valor crítico superior: {f_crit_upper}")
print(f"Valor crítico inferior: {f_crit_lower}")


Dado que el estadístico de prueba es mayor al valor crítico superior, rechazamos la hipótesis nula, lo que indica que hay una probabilidad menor al 0.1% de que las varianzas sean homogéneas (dadas estas muestras para los grupos).

Ahora sí podemos elegir nuestro estadístico, que será:

$$t=\frac{(\overline{x_1}-\overline{x_2})}{\sqrt{\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}}}$$

El cual sigue una distribución t de student con:
- desviación estándar $s=\sqrt{s_1^2/n_1+s_2^2/n_2}$;
- media $\mu=0$;
- grados de libertad $$\nu = \frac{\left(\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}\right)^2}{\frac{\left(\frac{s_1^2}{n_1}\right)^2}{n_1-1} + \frac{\left(\frac{s_2^2}{n_2}\right)^2}{n_2-1}}$$.


Realicemos ahora el test para ver si podemos decir que la media del grupo A es superior a la del grupo B con significancia \(\alpha = 0.001\). Al igual que antes, nuestra hipótesis nula será que ambas medias son iguales, i.e.
$$H_0: \mu_A - \mu_B = 0$$
Las hipótesis alternativas que consideraremos serán:
1. que la media salarial del grupo A es distinta a la del grupo B, i.e., $$H^1_a: \mu_A - \mu_B > 0 \rightarrow RR_1 = \{ t \geq t_{\alpha/2, \nu} \vee t \leq -t_{\alpha/2, \nu} \}$$
2. que la media salarial del grupo A es superior a la del grupo B, i.e., $$H^2_a: \mu_A - \mu_B > 0 \rightarrow RR_2 = \{ t \geq t_{\alpha, \nu} \}$$


In [ ]:
# Test de Welch

# Calculo de estadístico de prueba
t = (x1 - x2) / np.sqrt(s1_sqrd / n1 + s2_sqrd / n2)

# Grados de libertad
dof = ((s1_sqrd / n1 + s2_sqrd / n2) ** 2) / ((s1_sqrd / n1) ** 2 / (n1 - 1) + (s2_sqrd / n2) ** 2 / (n2 - 1))

# t-critical values
t_crit_neq = stats.t.ppf(1-alpha / 2, dof)
t_crit_great = stats.t.ppf(1-alpha, dof)
# se ingresa 1-alpha pues la sintaxis de la función
# pide como primer argumento q=1-alpha

print(f"Estadístico de prueba: {t}")
print(f"Ha1: mu_A != mu_B, RR=(t>= {t_crit_neq} o t <= {-t_crit_neq})")
print(f"Ha2: mu_A > mu_B, RR=(t>= {t_crit_great})")


Dado que $t \geq t_{1-\frac{\alpha}{2},\nu}$, se rechaza la hipótesis nula en ambos casos. Esto indica que:
1. hay una probabilidad menor al 0.1% de que las medias de las distribuciones coincidan (dadas estas muestras para los grupos).
2. hay una probabilidad menor al 0.1% de que la media del grupo A no sea mayor que la media del grupo B (dadas estas muestras para los grupos).

Concluimos entonces que los hombres cisgénero heterosexuales ganan más que el resto, con un nivel de significancia $\alpha = 0.001$.

### 2.2 P-valor

1. Calcule el p-valor y decida si rechazar o no la hipótesis nula.
2. Interprete el resultado.

Links útiles:
* [Test de hipótesis usando scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
* [Test de Welch](http://daniellakens.blogspot.com/2015/01/always-use-welchs-t-test-instead-of.html)

El valor p representa la probabilidad de observar los datos de la muestra (o datos más extremos) asumiendo que la hipótesis nula es verdadera. Por lo que, si el valor p es menor que el nivel de significancia, entonces se rechaza la hipótesis nula con un nivel de significancia de alpha.

In [ ]:
# Calculo de p-value

p_value = 2 * stats.t.sf(t, dof)
print(f"p-value: {p_value}")

Dado que p-value < $\alpha$, se rechaza Ho, es extremadamente improbable que se haya obtenido una diferencia de medias tan grande si las medias poblacionales fueran iguales. Por lo tanto, se puede concluir que las medias poblacionales son diferentes.

### [Opcional] 2.3 Potencia del test

Nuestra muestra, ¿era lo suficientemente grande para detectar si existe o no una diferencia entre los grupos?

1. Utilice la función `tt_ind_solve_power` para calcular el tamaño necesario de la muestra para un poder estadístico de 0.8, 0.9 y 0.95, asumiendo una significancia estadística de 0.05.
2. ¿Cómo intepretan el poder estadístico de un test? Dado su conocimiento de dominio sobre los datos, ¿les parece que esta muestra es lo suficientemente grande para ser representativo de la tendencia general? ¿y para utilizarlo en un juicio penal contra una empresa XX por una causa de discriminación?

[Documentación](https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html)

NOTA: este análisis debería hacerse ANTES de recolectar los datos.

In [ ]:
from statsmodels.stats.power import tt_ind_solve_power

In [ ]:
effect_size = (groupA.mean() - groupB.mean()) / groupB.std()
# nobs1=None  - What we want to know
alpha = 0.05
ratio = len(groupB) / len(groupA)
power = 0.8
tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=ratio)

In [ ]:
effect_size = (groupA.mean() - groupB.mean()) / groupB.std()
# nobs1=None  - What we want to know
alpha = 0.05
ratio = len(groupB) / len(groupA)
power = 0.9
tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=ratio)

In [ ]:
effect_size = (groupA.mean() - groupB.mean()) / groupB.std()
# nobs1=None  - What we want to know
alpha = 0.05
ratio = len(groupB) / len(groupA)
power = 0.95
tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=ratio)

Esto nos dice que para asegurar una probabilidad de 0.8, 0.9 y 0.95 de rechazar correctamente la hipótesis nula (en caso de que la alternativa sea cierta), necesitamos tener un número de muestras del primer grupo (`groupA`, `"Varón cis"`) de al menos 199, 266 y 328, respectivamente.

## Ejercicio 3: Comunicación y visualización

**Consigna:** Seleccionen un resultado que les parezca relevante a partir de alguno de los ejercicios del entregable. Diseñe e implemente una comunicación en base a este mensaje, en un archivo PDF.

Elija las palabras y visualización más adecuada para que la comunicación sea entendible, efectiva y se dapte a UNA de las siguientes situaciones:

1. Una sección en un artículo de difusión a presentar como parte de una organización sin fines de lucro.
No más de 1 página A4 (o dos si los gráficos son muy grandes).
    1. Ejemplo: Alguna de las secciones [Los ecosistemas de emprendimiento de América Latina y el Caribe frente al COVID-19: Impactos, necesidades y recomendaciones](https://publications.iadb.org/es/los-ecosistemas-de-emprendimiento-de-america-latina-y-el-caribe-frente-al-covid-19-impactos-necesidades-y-recomendaciones), por ejemplo la sección *2.2. Reacciones de los emprendedores*.
    2. Ejemplo: Alguna de las secciones de [The state of gender pay gap in 2021](https://www.payscale.com/data/gender-pay-gap?tk=carousel-ps-rc-job)
    3. Puntos clave: 
        1. Simpleza de los gráficos.
        2. Comunicación en lenguaje simple a personas que no son necesariamente expertos de dominio.
        2. Selección de UNA oración sobre la que se hace énfasis.
        3. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

2. Una publicación científica o reporte técnico interno. No más de una página A4:
    2. Ejemplo: La sección de resultados de [IZA DP No. 12914: The Impact of a Minimum Wage Change on the Distribution of Wages and Household Income](https://www.iza.org/publications/dp/12914/the-impact-of-a-minimum-wage-change-on-the-distribution-of-wages-and-household-income).
    2. Ejemplo: Alguna de las secciones de [Temporary reduction in daily global CO2 emissions during the COVID-19 forced confinement](https://www.nature.com/articles/s41558-020-0797-x)
    3. Puntos clave:
        3. Nivel de detalle técnico requerido. Es necesario justificar la validez del análisis.
        4. La idea presentada puede ser más compleja. Pueden asumir que la audiencia tiene conocimiento técnico y va a analizar las visualizaciones en detalle. 
        5. Pueden presentar más en detalle las limitaciones del análisis (significancia estadística, etc.)
        2. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

3. Un tweet (o post de LinkedIn) para la cuenta de su empresa consultora que hace análisis de datos. El objetivo es promocionar un análisis de datos abiertos que van a incluir en su portfolio:
    1. Ejemplo: [Comparación vacunas covid](https://twitter.com/infobeautiful/status/1381577746527236098?s=20)
    2. Ejemplo: [Tweet del BID](https://twitter.com/el_BID/status/1388508583944507396?s=20). Lo valioso de este tweet es que usaron un único número para transmitir un mensaje. Puede ser algo así, o con un gráfico muy simple.
    3. Ejemplo: [Cambio climático](https://twitter.com/UNFCCC/status/1387732156190011394?s=20) Es un muy buen ejemplo, excepto que el gráfico no se lee nada y hay que entrar a la publicación original.
    3. Ejemplo: [¿Cuánto están los programadores en las empresas?](https://www.linkedin.com/posts/denis-rothman-0b034043_tech-career-work-activity-6793861923269054464-gS6y) (No verificamos la veracidad o seriedad de la fuente).
    4. Puntos clave:
        1. Su audiencia no va a mirar la visualización por más de unos segundos, y no tiene conocimiento técnico.
    3. Tienen que incluir además una *breve* descripción de cómo obtuvieron los datos que están presentando, que no entraría en el tweet.


Elegimos hacer el 3. en la carpeta esta adjuntada la imagen del tweet. 